In [88]:
import requests
import pandas as pd
import pyodbc

In [89]:
def ticker():
    conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=MSI\\SQLEXPRESS;'
                      'Database=dataFinance;'
                      'Trusted_Connection=yes;')

    cursor = conn.cursor()

    sql_query = "SELECT DISTINCT ticker FROM Info;"

    cursor.execute(sql_query)

    tickers = [row.ticker for row in cursor]

    conn.close()

    return tickers

In [90]:
def ratio(ticker):  
    
    api_url = f"https://apipubaws.tcbs.com.vn/tcanalysis/v1/finance/{ticker}/financialratio?yearly=0&isAll=true"

    response = requests.get(api_url)

    if response.status_code == 200:
        data = response.json()

        df = pd.DataFrame(data)
        return df
    else:
        print("Yêu cầu không thành công. Mã trạng thái:", response.status_code)
        return None
    

In [91]:
import pyodbc
import pandas as pd

def insert_Ratio(df):
    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=MSI\\SQLEXPRESS;'
                          'Database=dataFinance;'
                          'Trusted_Connection=yes;')
    
    cursor = conn.cursor()

    df = df.replace('-', 0)

    df = df.fillna(0)

    def custom_float_conversion(value):
        if isinstance(value, str):
            try:
                return float(value.replace(',', ''))
            except ValueError:
                return value
        elif isinstance(value, int):
            return float(value)
        else:
            return value

    columns_to_convert = [
        'priceToEarning', 'priceToBook', 'valueBeforeEbitda', 'dividend', 'roe', 'roa', 'daysReceivable',
        'daysInventory', 'daysPayable', 'ebitOnInterest', 'earningPerShare', 'bookValuePerShare', 'interestMargin',
        'nonInterestOnToi', 'badDebtPercentage', 'provisionOnBadDebt', 'costOfFinancing', 'equityOnTotalAsset',
        'equityOnLoan', 'costToIncome', 'equityOnLiability', 'currentPayment', 'quickPayment', 'epsChange',
        'ebitdaOnStock', 'grossProfitMargin', 'operatingProfitMargin', 'postTaxMargin', 'debtOnEquity', 'debtOnAsset',
        'debtOnEbitda', 'shortOnLongDebt', 'assetOnEquity', 'capitalBalance', 'cashOnEquity', 'cashOnCapitalize',
        'cashCirculation', 'revenueOnWorkCapital', 'capexOnFixedAsset', 'revenueOnAsset', 'postTaxOnPreTax',
        'ebitOnRevenue', 'preTaxOnEbit', 'preProvisionOnToi', 'postTaxOnToi', 'loanOnEarnAsset', 'loanOnAsset',
        'loanOnDeposit', 'depositOnEarnAsset', 'badDebtOnAsset', 'liquidityOnLiability', 'payableOnEquity',
        'cancelDebt', 'ebitdaOnStockChange', 'bookValuePerShareChange', 'creditGrowth'
    ]

    # Chuyển các cột thành kiểu float
    df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

    sql_queries = []  # Tạo một danh sách các câu lệnh SQL

    for index, row in df.iterrows():
        sql_query = f"INSERT INTO ratio (" \
                    f"ticker, quarter, year, priceToEarning, priceToBook, valueBeforeEbitda, dividend, roe, roa, " \
                    f"daysReceivable, daysInventory, daysPayable, ebitOnInterest, earningPerShare, bookValuePerShare, " \
                    f"interestMargin, nonInterestOnToi, badDebtPercentage, provisionOnBadDebt, costOfFinancing, " \
                    f"equityOnTotalAsset, equityOnLoan, costToIncome, equityOnLiability, currentPayment, quickPayment, " \
                    f"epsChange, ebitdaOnStock, grossProfitMargin, operatingProfitMargin, postTaxMargin, debtOnEquity, " \
                    f"debtOnAsset, debtOnEbitda, shortOnLongDebt, assetOnEquity, capitalBalance, cashOnEquity, " \
                    f"cashOnCapitalize, cashCirculation, revenueOnWorkCapital, capexOnFixedAsset, revenueOnAsset, " \
                    f"postTaxOnPreTax, ebitOnRevenue, preTaxOnEbit, preProvisionOnToi, postTaxOnToi, loanOnEarnAsset, " \
                    f"loanOnAsset, loanOnDeposit, depositOnEarnAsset, badDebtOnAsset, liquidityOnLiability, payableOnEquity, " \
                    f"cancelDebt, ebitdaOnStockChange, bookValuePerShareChange, creditGrowth) " \
                    f"VALUES ('{row['ticker']}', '{row['quarter']}', {row['year']}, {row['priceToEarning']}, " \
                    f"{row['priceToBook']}, {row['valueBeforeEbitda']}, {row['dividend']}, {row['roe']}, {row['roa']}, " \
                    f"{row['daysReceivable']}, {row['daysInventory']}, {row['daysPayable']}, {row['ebitOnInterest']}, " \
                    f"{row['earningPerShare']}, {row['bookValuePerShare']}, {row['interestMargin']}, {row['nonInterestOnToi']}, " \
                    f"{row['badDebtPercentage']}, {row['provisionOnBadDebt']}, {row['costOfFinancing']}, " \
                    f"{row['equityOnTotalAsset']}, {row['equityOnLoan']}, {row['costToIncome']}, {row['equityOnLiability']}, " \
                    f"{row['currentPayment']}, {row['quickPayment']}, {row['epsChange']}, {row['ebitdaOnStock']}, " \
                    f"{row['grossProfitMargin']}, {row['operatingProfitMargin']}, {row['postTaxMargin']}, " \
                    f"{row['debtOnEquity']}, {row['debtOnAsset']}, {row['debtOnEbitda']}, {row['shortOnLongDebt']}, " \
                    f"{row['assetOnEquity']}, {row['capitalBalance']}, {row['cashOnEquity']}, {row['cashOnCapitalize']}, " \
                    f"{row['cashCirculation']}, {row['revenueOnWorkCapital']}, {row['capexOnFixedAsset']}, " \
                    f"{row['revenueOnAsset']}, {row['postTaxOnPreTax']}, {row['ebitOnRevenue']}, {row['preTaxOnEbit']}, " \
                    f"{row['preProvisionOnToi']}, {row['postTaxOnToi']}, {row['loanOnEarnAsset']}, {row['loanOnAsset']}, " \
                    f"{row['loanOnDeposit']}, {row['depositOnEarnAsset']}, {row['badDebtOnAsset']}, {row['liquidityOnLiability']}, " \
                    f"{row['payableOnEquity']}, {row['cancelDebt']}, {row['ebitdaOnStockChange']}, {row['bookValuePerShareChange']}, " \
                    f"{row['creditGrowth']})"
        sql_queries.append(sql_query)  # Thêm câu lệnh SQL vào danh sách

    for sql_query in sql_queries:
        cursor.execute(sql_query)  # Thực hiện từng câu lệnh SQL

    # Thực hiện commit để lưu thay đổi
    conn.commit()
    
    # Đóng kết nối
    conn.close()


In [92]:
def main():
    tickers = ticker()
    data_list = []

    for ticker_symbol in tickers:
        result = ratio(ticker_symbol)
        if result is not None:
            data_list.append(result)

    if data_list:
        df = pd.concat(data_list, ignore_index=True)
    else:
        print("Không có dữ liệu hợp lệ")
    insert_Ratio(df)
    print("Insert success")
    
    
    

In [93]:
if __name__ == '__main__':
    main()


Insert success
